# 設定ファイルの編集 -- slurm.conf

---

OpenHPC環境の設定ファイルの内容を変更します。

## 概要

このNotebookでは、VCノードの`slurm.conf`の内容を変更し、その変更を反映する手順を示します。

![処理の流れ](images/ohpc-010.png)

主な手順は以下のようになります。

1. マスターノードの設定ファイル`slurm.conf`をNotebook環境に取得する
2. 取得したファイルのバックアップを作成する
3. Notebookの編集機能を利用して設定ファイルの変更をおこなう
4. 変更した設定ファイルを各VCノードに配置する
5. 各VCノードのSLURMデーモンに設定ファイルの再読み込みを指示する

## 前提条件

このNotebookを実行するための前提条件を満たしていることを確認します。

以下のことを前提条件とします。
* 構築済のOpenHPC環境がある
* 各VCノードに対してAnsibleで操作できるように設定されている
* 各VCノードに対して管理者権限で操作が行える

VCノードを作成時に指定した値を確認するために `group_vars` ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

操作対象となる UnitGroup 名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'OpenHPC'

ugroup_name = 

疎通確認を行います。

In [ ]:
!ansible {ugroup_name} -m ping 

管理者権限でコマンドが実行できることを確認します。

In [ ]:
!ansible {ugroup_name} -b -a 'whoami'

## パラメータの設定

### 編集対象となるファイルの指定

対象となるファイルのパスを指定してください。

In [ ]:
target_file = '/etc/slurm/slurm.conf'

## 設定ファイルの編集

OpenHPC環境の設定ファイルを取得して、Jupyter Notebookの編集機能を用いて設定ファイルを編集します。

次のセルを実行すると、以下の手順が実行されます。

1. マスターノードの設定ファイル(`slurm.conf`)をローカル環境に取得する
2. 取得した設定ファイルのバックアップを作成する
3. Jupyter Notebookの編集機能を利用して設定ファイルを編集するためのリンクを表示する

表示されたリンクをクリックして、設定ファイルを編集してください。

ローカル環境に取得したファイルは、以下のパスに格納されています。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/slurm.conf`

`{ugroup_name}` には UnitGroup名が、`{YYYYMMDDHHmmssfffff}` にはファイルを取得したタイムスタンプが入ります。

また、バックアップファイルは以下のパスに格納されます。

`./edit/{ugroup_name}/{YYYYMMDDHHmmssffffff}/slurm.conf.orig`

In [ ]:
%run scripts/edit_conf.py
fetch_conf(target_file, ugroup_name, 'master')

上のセルに表示されているリンクをクリックするとファイルの編集画面が開きます。必要な箇所の編集を行い、その結果をファイルに保存してください。ファイルの保存を行うにはメニューの File -- Save を選択するか、キーボードで Ctrl-S をタイプしてください。

`slurm.conf`の設定方法の詳細については [slurm.conf(5)](https://slurm.schedmd.com/slurm.conf.html)を参照してください。

また、次のセルを実行すると計算ノードの実際のリソース量を確認することができます。`slurm.conf`を編集する際の参考にしてください。通常は次のセルの実行結果に合わせて `slurm.conf` の `NodeName` の行を以下のように更新することになります。

```
NodeName=c[1-4] CPUs=4 Boards=1 SocketsPerBoard=1 CoresPerSocket=2 ThreadsPerCore=2 RealMemory=15751 State=UNKNOWN
```

In [ ]:
!ansible {ugroup_name}_compute -a '/usr/sbin/slurmd -C'

設定ファイルの編集後に次のセルを実行すると、編集の前後での差分を確認することができます。

In [ ]:
%run scripts/edit_conf.py
show_local_conf_diff(target_file, ugroup_name, 'master')

## 編集した設定ファイルの反映

編集したファイルをVCノードに配置して、設定ファイルの変更内容をコンテナに反映させます。

### 編集内容の反映

前章で編集した設定ファイル `slurm.conf` を各VCノードに配置します。

まず、変更前の状態を確認しておきます。

In [ ]:
!ansible {ugroup_name}_master -a 'scontrol show node'

では、実際に設定ファイルの変更を行います。

次のセルを実行すると、以下の手順が実行されます。

1. 編集前と編集後の設定ファイルの差分を表示する
2. 編集した設定ファイル(`slurm.conf`)を各VCノードに配置する

In [ ]:
%run scripts/edit_conf.py
upload_conf(target_file, ugroup_name)

マスターノード、計算ノードの SLURMデーモンに設定ファイルの再読み込みを指示します。

In [ ]:
!ansible {ugroup_name}_master -b -a 'systemctl reload slurmctld'
!ansible {ugroup_name}_compute -b -a 'systemctl reload slurmd'

設定ファイル変更後のノードの状態を確認します。

In [ ]:
!ansible {ugroup_name}_master -a 'scontrol show node'

設定ファイルに誤りがあり、SLURMデーモンが停止してしまった場合は「4. 設定ファイルの編集」以降のセルを unfreeze してから再度実行してください。また、SLURMデーモンが停止している場合は `systemctl reload` のかわりに `systemctl restart` を指定してサービスの再起動を行ってください。